In [8]:
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
import operator
import csv
from csv import writer

In [9]:
df = pd.read_csv('nbaDatabasePreProcessed2004-2024.csv')
df

,date,homeTeam,homeTeam_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,...,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season,isPlayoffGame
0,2024-06-17,Boston Celtics,BOS,0.797980,79,20,106,28,39,19,...,28,35,18,4,4,13,20,Boston Celtics,2023-2024,True
1,2024-06-14,Dallas Mavericks,DAL,0.613861,62,39,122,34,27,31,...,27,31,18,2,5,13,19,Dallas Mavericks,2023-2024,True
2,2024-06-12,Dallas Mavericks,DAL,0.610000,61,39,99,31,20,19,...,30,36,26,4,6,9,19,Boston Celtics,2023-2024,True
3,2024-06-09,Boston Celtics,BOS,0.802083,77,19,105,25,29,29,...,34,43,21,5,3,15,17,Boston Celtics,2023-2024,True
4,2024-06-06,Boston Celtics,BOS,0.800000,76,19,107,37,26,23,...,33,43,9,8,1,11,16,Boston Celtics,2023-2024,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,0.000000,0,0,104,21,19,27,...,36,47,24,9,5,13,28,Indiana Pacers,2004-2005,False
25727,2004-11-03,Boston Celtics,BOS,0.000000,0,0,95,22,31,30,...,27,41,20,9,2,16,26,Philadelphia 76ers,2004-2005,False
25728,2004-11-02,Los Angeles Lakers,LAL,0.000000,0,0,89,24,26,22,...,32,48,17,10,8,16,21,Los Angeles Lakers,2004-2005,False
25729,2004-11-02,Detroit Pistons,DET,0.000000,0,0,87,19,18,22,...,27,36,8,4,5,16,24,Detroit Pistons,2004-2005,False


In [10]:
#maps franchises to subjectID
subjectID_dict = {
    "ATL": 1,
    "BOS": 2,
    "BKN": 3,
    "CHA": 4,
    "CHI": 5,
    "CLE": 6,
    "DAL": 7,
    "DEN": 8,
    "DET": 9,
    "GSW": 10,
    "HOU": 11,
    "IND": 12,
    "LAC": 13,
    "LAL": 14,
    "MEM": 15,
    "MIA": 16,
    "MIL": 17,
    "MIN": 18,
    "NOP": 19,
    "NYK": 20,
    "OKC": 21,
    "ORL": 22,
    "PHI": 23,
    "PHX": 24,
    "POR": 25,
    "SAC": 26,
    "SAS": 27,
    "TOR": 28,
    "UTA": 29,
    "WAS": 30,
    "NOH": 19, #New Orleans Hornets is same franchise as NOP- New Orleans Pelicans
    "NJN": 3, #New Jersey Nets relocated to Brooklyn
    "SEA": 21, #Seattle Supersonics Relocated to OKC
    "NOK": 19, #New Orleans Hornets relocated to OKC 2005-2007 - Eventually became New Orleans Pelicans
}

In [11]:
#map the subject IDs to a number to handle franchise moves
df["homeTeamSubject_id"] = df["homeTeam_id"].map(subjectID_dict)
df["awayTeamSubject_id"] = df["awayTeam_id"].map(subjectID_dict)

cols = df.columns.tolist()
cols.insert(cols.index("homeTeam_id") + 1, cols.pop(cols.index("homeTeamSubject_id")))
cols.insert(cols.index("awayTeam_id") + 1, cols.pop(cols.index("awayTeamSubject_id")))
df = df[cols]

df

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,...,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season,isPlayoffGame
0,2024-06-17,Boston Celtics,BOS,2,0.797980,79,20,106,28,39,...,28,35,18,4,4,13,20,Boston Celtics,2023-2024,True
1,2024-06-14,Dallas Mavericks,DAL,7,0.613861,62,39,122,34,27,...,27,31,18,2,5,13,19,Dallas Mavericks,2023-2024,True
2,2024-06-12,Dallas Mavericks,DAL,7,0.610000,61,39,99,31,20,...,30,36,26,4,6,9,19,Boston Celtics,2023-2024,True
3,2024-06-09,Boston Celtics,BOS,2,0.802083,77,19,105,25,29,...,34,43,21,5,3,15,17,Boston Celtics,2023-2024,True
4,2024-06-06,Boston Celtics,BOS,2,0.800000,76,19,107,37,26,...,33,43,9,8,1,11,16,Boston Celtics,2023-2024,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,6,0.000000,0,0,104,21,19,...,36,47,24,9,5,13,28,Indiana Pacers,2004-2005,False
25727,2004-11-03,Boston Celtics,BOS,2,0.000000,0,0,95,22,31,...,27,41,20,9,2,16,26,Philadelphia 76ers,2004-2005,False
25728,2004-11-02,Los Angeles Lakers,LAL,14,0.000000,0,0,89,24,26,...,32,48,17,10,8,16,21,Los Angeles Lakers,2004-2005,False
25729,2004-11-02,Detroit Pistons,DET,9,0.000000,0,0,87,19,18,...,27,36,8,4,5,16,24,Detroit Pistons,2004-2005,False


In [12]:
operations = {
    'Team_points_total': operator.ge,
    'Team_points_q1': operator.ge,
    'Team_points_q2': operator.ge,
    'Team_points_q3': operator.ge,
    'Team_points_q4': operator.ge,
    'Team_FG': operator.ge,
    'Team_FG_made': operator.ge,
    'Team_3P': operator.ge,
    'Team_O_Reb': operator.ge,
    'Team_D_Reb': operator.ge,
    'Team_Total_Reb': operator.ge,
    'Team_Ast': operator.ge,
    'Team_Stl': operator.ge,
    'Team_Blk': operator.ge,
    'Team_TO': operator.le,
    'Team_PF': operator.le

}

keyList = list(operations.keys())
print(keyList)

['Team_points_total', 'Team_points_q1', 'Team_points_q2', 'Team_points_q3', 'Team_points_q4', 'Team_FG', 'Team_FG_made', 'Team_3P', 'Team_O_Reb', 'Team_D_Reb', 'Team_Total_Reb', 'Team_Ast', 'Team_Stl', 'Team_Blk', 'Team_TO', 'Team_PF']


In [16]:
for i in range(len(keyList)):
    #populate dfIDtoStat - should be double the size of df
    homedf = df[['date', 'homeTeamSubject_id', f'home{keyList[i]}']]
    homedf.columns = ['date', 'team_id', f'{keyList[i]}']
    awaydf = df[['date', 'awayTeamSubject_id', f'away{keyList[i]}']]
    awaydf.columns = ['date', 'team_id', f'{keyList[i]}']
    df['predicted_winner'] = np.nan
    scores = []


    dfIDtoStat = pd.concat([homedf, awaydf], ignore_index=True)
    #sort by date
    dfIDtoStat = dfIDtoStat.sort_values(by='date', ascending=False).reset_index(drop=True)
    for index, row in df.iterrows():
        try:
            #get all games that occurred before game
            homeTeam_id = row['homeTeamSubject_id']
            awayTeam_id = row['awayTeamSubject_id']
            date = row['date']
            #limit dfWindow to include rows that occured before date of row we are on
            dfWindow = dfIDtoStat[dfIDtoStat['date'] < date]
            #limit dfWindow to include only rows of each team for prediction, take highest index as it descends by date and hold the amount of points they scored
            homeTeam_performance_prev_game = dfWindow[dfWindow['team_id'] == homeTeam_id].reset_index(drop=True).iloc[0][f'{keyList[i]}']
            awayTeam_performance_prev_game = dfWindow[dfWindow['team_id'] == awayTeam_id].reset_index(drop=True).iloc[0][f'{keyList[i]}']
            #predict the team who performed better in given statistic based on operation
            if operations[keyList[i]](homeTeam_performance_prev_game, awayTeam_performance_prev_game):
                df.at[index, 'predicted_winner'] = row['homeTeam']
            else:
                df.at[index, 'predicted_winner'] = row['awayTeam']
        except:
            pass

        #drop nans from bottom of df
        df = df.dropna()
        score = accuracy_score(df['winner'], df['predicted_winner'])
        scores.append(f'{keyList[i]}')
        scores.append(score)
        

        with open('accuracyScores2.csv', 'a', encoding='UTF8', newline='') as accuracyScores:
            writer = csv.writer(accuracyScores)      
            writer.writerows(scores) 

C:\Users\Max Braithwaite\AppData\Local\Temp\ipykernel_13640\440620064.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_winner'] = np.nan
c:\Users\Max Braithwaite\anaconda3\envs\cvenv\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Max Braithwaite\anaconda3\envs\cvenv\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Error: iterable expected, not numpy.float64